In [1]:
import rioxarray as rx
import geopandas as gpd
from pathlib import Path
import requests as r
import pandas as pd

data_path = Path("data-fire/imagery")
out_path = Path("data-fire/processed")
img_paths = list(data_path.glob("*.img"))

This throws out about 4 fires out of >45 that don't conform to the same filename type

In [2]:
metadata = sorted([str(path.name).split("_")[:-1] for path in img_paths if len(str(path).split("_")) == 4]) # getting metadata from filenames
df = pd.DataFrame(metadata, columns=['site', 'date', 'product'])
df["datetime"] = pd.to_datetime(df['date'])
df.index = df['datetime']
df = df.sort_index()

In [8]:
df

,site,date,product,datetime
datetime,,,,
2002-06-08,wolf,20020608,30m,2002-06-08
2002-06-10,wolf,20020610,barc,2002-06-10
2002-08-27,mcnally,20020827,30m,2002-08-27
2003-09-19,deer,20030919,30m,2003-09-19
2003-09-19,grindstone,20030919,30m,2003-09-19
...,...,...,...,...
2010-10-12,sheep,20101012,barc4,2010-10-12
2011-08-19,lion,20110819,post,2011-08-19
2011-08-19,lion,20110819,dnbr,2011-08-19


In [9]:
img_list = []
for i in img_paths:
    x = rx.open_rasterio(i)
    x.name = i.name
    img_list.append(x)

In [ ]:
nlcd_2016 = rx.open_rasterio(data_path / "external/")

Need to check the present day (2016 NLCD) land cover classes for each burned area to make sure it has forest coverage type. Use this filter for selection before downloading gEDI data and store the majority land cover class within the df. 

In [ ]:
for i in img_list:
    i.rio.reproject(4326).rio.to_raster(str(out_path / Path("WGS84_"+str(i.name))))

In [ ]:
def bounds_request(bounds):
    """
    Requests GEDI L2B data download links using the GEDI Finder service.
    
    https://lpdaacsvc.cr.usgs.gov/services/gedifinder
    
    Args:
        bounds returned from rioxarray rAster Array.
        It's unpacked to ul_lat,ul_lon,lr_lat,lr_lon
    """
    ul_lon = bounds[0]
    ul_lat = bounds[-1]
    lr_lon = bounds[2]
    lr_lat = bounds[1]
    result = r.get(f"https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product=GEDI01_B&version=001&bbox=[{ul_lat},{ul_lon},{lr_lat},{lr_lon}]")
    print(result)
    return result.json()['data']
    

In [45]:
all_urls = []
for i in img_list:
    url_list = bounds_request(i.rio.bounds())
    if len(url_list) > 0:
        all_urls.extend(url_list)

<Response [200]>


In [47]:
with open("fire_gedi_url_list.txt", 'w') as output:
    for row in all_urls:
        output.write(str(row) + '\n')